# Chapter 2: Working with text data

## 2.2: Tokenizing Text

In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
print("Total number of characters: ", len(raw_text))

print(raw_text[:99])

In [ ]:
import re
text = "Hello, world. This, is a test."

# split on whitespace
result = re.split(r'(\s)',text)
print(result)

#split on whitespace, commas, and periods
result = re.split(r'([,.]|\s)', text)
print(result)

#remove whitespace characters
result = [item for item in result if item.strip()]
print(result)

In [ ]:
# build a more advanced tokenizer
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

In [ ]:
# apply our more advanced (but still very basic!) tokenizer to Edith Wharton's entire short story
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]

## 2.3 Converting tokens into token IDs

In [ ]:
# create a list of all unqiue tokens (words) and sort alphabetically. This is our vocabulary.
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

In [ ]:
# Display the first 51 tokens in our vocabulary.
vocab = {token:integer for integer,token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

In [ ]:
# A class that handles this tokenization, creation of the vocabulary, and an integer-to-string lookup on our vocabulary.
class SimpleTokenizerV1:
    def __init__(self,vocab):
        # store the vocabulary as a class attribute for access in encode/decode methods
        self.str_to_int = vocab
        # create an inverse vocabulary that maps token IDs back to the original text tokens
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        # process input text into token IDs
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        # convert ids back into tokens
        text = " ".join([self.int_to_str[i] for i in ids])

        # removes spaces before specified punctuation
        text = re.sub(r'\s+([,.?!"()\'])', r'\1',text)
        return text

In [ ]:
# Initialize our tokenizer with the full vocab of the entire story.
tokenizer = SimpleTokenizerV1(vocab)

# tokenize a passage from Edith Wharton's short story
text = """"It's the last he painted, you know,"
        Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

In [ ]:
# Can we turn these IDs back into text?
print(tokenizer.decode(ids))

In [ ]:
# Try to tokenize text not in the training set used to generate our vocabulary
text = "Hello, do you like tea?"
print(tokenizer.encode(text))

# Results in KeyError

## 2.4 Adding special context tokens

In [ ]:
# Add a token to represent the end of a text and one that represents an unknown token
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer,token in enumerate(all_tokens)}

print(len(vocab.items()))

# print the last 5 items in our vocabulary, which should include our new special tokens
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

In [ ]:
# Update our tokenizer to handle our new special tokens
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]

        # Replaces unknown words with the <|unk|> token
        preprocessed = [item if item in self.str_to_int
                       else "<|unk|>" for item in preprocessed]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])

        # removes spaces before specified punctuation
        text = re.sub(r'\s+([,.?!"()\'])', r'\1',text)
        return text

In [ ]:
# Generate two unrelated texts and concatenate them with our special end of text token
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1,text2))
print(text)

In [ ]:
# Tokenize our conatenated texts
tokenizer = SimpleTokenizerV2(vocab)
print(tokenizer.encode(text))
print(tokenizer.decode(tokenizer.encode(text)))

## 2.5 Byte pair encoding

In [ ]:
from importlib.metadata import version
import tiktoken
print("tiktoken version: ", version("tiktoken"))

In [ ]:
# Instantiate the bye pair encoding (BPE) tokenizer
tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
# Encode our text using tiktoken
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
    "of someunknownPlace."
)
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

# Decode our tokenized string using tiktoken
strings = tokenizer.decode(integers)
print(strings)

### Exercise 2.1 Byte pair encoding of unknown words

In [ ]:
unknown_words_text = "Akwirw ier"
encoded_unknowns = tokenizer.encode(unknown_words_text)
print(encoded_unknowns)

for encoded_int in encoded_unknowns:
    print(tokenizer.decode([encoded_int]))

print(tokenizer.decode(encoded_unknowns))

## 2.6 Data sampling with a sliding window

In [ ]:
# Load and encode the entire text of "The Verdict" short story
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

In [ ]:
# Remove the first 50 tokens from the dataset for demonstration purposes
enc_sample = enc_text[50:]

In [ ]:
context_size = 4 #how many tokens are included in the input
x = enc_sample[:context_size] # Inputs
y = enc_sample[1:context_size+1] # Targets

# The targets are the inputs, shifted by 1 position
print(f"x: {x}")
print(f"y:      {y}")

In [ ]:
# Create a next-word predicition task

for i in range(1, context_size + 1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->", desired)

In [ ]:
# Do the same thing, but decode the tokens
for i in range(1, context_size + 1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

Note to self: Is this interesting? It seems like I'm just encoding a source text, and then regurgitating the same text through tokens. Am I missing something?

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt) # Tokenizes the entire text

        # uses a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self): # Returns the total number of rows in the dataset
        return len(self.input_ids)

    def __getitem__(self, idx): # Returns a single row from the dataset.
        return self.input_ids[idx], self.target_ids[idx]

In [ ]:
def create_dataloader_v1(txt, batch_size=4, max_length=246,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last, # True drops the last batch if it is shorter than batch_size to prevent loss_spikes during training
        num_workers=num_workers # number of CPU processes to use for preprocessing
    )

    return dataloader

In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader) # Converts the dataloader into a Python iterator to fetch the next entry via Python's built in next() function
first_batch = next(data_iter)
print(first_batch)

In [ ]:
second_batch = next(data_iter)
print(second_batch)

## Exercise 2.2 Data Loaders with different strides and context sizes

In [ ]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=2, stride=2, shuffle=False)
data_iter = iter(dataloader) # Converts the dataloader into a Python iterator to fetch the next entry via Python's built in next() function
first_batch = next(data_iter)
print(first_batch)
second_batch = next(data_iter)
print(second_batch)
print('-'*20)

dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=8, stride=2, shuffle=False)
data_iter = iter(dataloader) # Converts the dataloader into a Python iterator to fetch the next entry via Python's built in next() function
first_batch = next(data_iter)
print(first_batch)
second_batch = next(data_iter)
print(second_batch)
print('-'*20)

dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=8, stride=1, shuffle=False)
data_iter = iter(dataloader) # Converts the dataloader into a Python iterator to fetch the next entry via Python's built in next() function
first_batch = next(data_iter)
print(first_batch)
second_batch = next(data_iter)
print(second_batch)

In [ ]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)